In [1]:
import pandas as pd
from googletrans import Translator
from tqdm import tqdm   # for progress bar

# 1. Load your Excel file
input_file = "lexicon_6000 words.xlsx"
df = pd.read_excel(input_file)

# 2. Initialize translator
translator = Translator()

# 3. Define new language columns (fixed codes)
new_langs = {
    "English": "en",
    "Zulu": "zu",
    "Xhosa": "xh",
    "Sepedi": "st",     # Sesotho (closest option available in Google Translate)
    "Shona": "sn",
    "Afrikaans": "af"
}

# Add empty columns if not already there
for lang in new_langs.keys():
    if lang not in df.columns:
        df[lang] = ""

# 4. Translation loop with progress bar
for i in tqdm(range(len(df)), desc="Translating words"):
    french_word = str(df.at[i, "French"])

    try:
        # Step A: French → English
        english_translation = translator.translate(french_word, src="fr", dest="en").text
        df.at[i, "English"] = english_translation

        # Step B: English → Target Languages
        for lang, code in new_langs.items():
            if lang == "English":
                continue  # already translated
            translation = translator.translate(english_translation, src="en", dest=code).text
            df.at[i, lang] = translation

    except Exception as e:
        print(f"⚠️ Error translating word '{french_word}': {e}")

# 5. Save expanded lexicon
output_file = "expanded_lexicon_6000.xlsx"
df.to_excel(output_file, index=False)

print(f"✅ Expanded lexicon saved as {output_file}")



ModuleNotFoundError: No module named 'pandas'

In [ ]:
import pandas as pd

# Load your expanded lexicon
df = pd.read_excel("expanded_lexicon_6000.xlsx")

# Find rows where English translation is missing
missing_rows = df[df["English"].isna() | (df["English"] == "")]

print(f"Found {len(missing_rows)} missing translations.")


Found 94 missing translations.


In [ ]:
from googletrans import Translator
import time

translator = Translator()

def safe_translate(text, src, dest, retries=3, delay=2):
    for _ in range(retries):
        try:
            result = translator.translate(text, src=src, dest=dest)
            if result and result.text:
                return result.text
        except Exception:
            time.sleep(delay)
    return None

# Only translate missing rows
for i, row in missing_rows.iterrows():
    french_word = str(row["French"]).strip()
    if not french_word:
        continue

    english_translation = safe_translate(french_word, src="fr", dest="en")
    if english_translation:
        df.at[i, "English"] = english_translation
        # Translate to other languages if missing
        for lang, code in {
            "Zulu": "zu", "Xhosa": "xh", "Sepedi": "st",
            "Shona": "sn", "Afrikaans": "af"
        }.items():
            if pd.isna(df.at[i, lang]) or df.at[i, lang] == "":
                df.at[i, lang] = safe_translate(english_translation, src="en", dest=code)
    else:
        print(f"⚠️ Could not translate '{french_word}'")

# Save fixed lexicon
df.to_excel("expanded_lexicon_fixed.xlsx", index=False)
print("✅ Missing translations fixed and saved!")


✅ Missing translations fixed and saved!


In [ ]:
import pandas as pd

# Load your expanded lexicon
df = pd.read_excel("expanded_lexicon_6000.xlsx")

# Find rows where English translation is missing
missing_rows = df[df["English"].isna() | (df["English"] == "")]

print(f"Found {len(missing_rows)} missing translations.")

Found 94 missing translations.


In [ ]:
missing_rows = df[df["English"].isna() | (df["English"] == "")]
print(f"Still missing after fixing: {len(missing_rows)}")

Still missing after fixing: 94


In [ ]:
import pandas as pd
from googletrans import Translator
import time

# Load your lexicon
df = pd.read_excel("expanded_lexicon_6000.xlsx")

# Find rows with missing English translations
missing_rows = df[df["English"].isna() | (df["English"] == "")]
print(f"Found {len(missing_rows)} missing translations.")

translator = Translator()

# Languages to translate to (fix Sepedi to nso)
langs = {
    "Zulu": "zu",
    "Xhosa": "xh",
    "Sepedi": "nso",   # Correct code if supported
    "Shona": "sn",
    "Afrikaans": "af"
}

def safe_translate(text, src, dest, retries=5, delay=5):
    """Translate text with retries and delay."""
    text = text.strip().replace("\u00A0", " ")  # Clean non-breaking spaces
    for _ in range(retries):
        try:
            result = translator.translate(text, src=src, dest=dest)
            if result and result.text:
                return result.text
        except Exception:
            time.sleep(delay)
    return None

# Process only missing rows
for i, row in missing_rows.iterrows():
    french_word = str(row["French"]).strip()
    if not french_word:
        continue

    english_translation = safe_translate(french_word, src="fr", dest="en")
    if english_translation:
        df.at[i, "English"] = english_translation
        for lang, code in langs.items():
            if pd.isna(df.at[i, lang]) or df.at[i, lang] == "":
                translated = safe_translate(english_translation, src="en", dest=code)
                df.at[i, lang] = translated if translated else ""
    else:
        print(f"⚠️ Could not translate '{french_word}'")

# Save the fixed lexicon
df.to_excel("expanded_lexicon_fixed_v2.xlsx", index=False)
print("✅ Missing translations fixed and saved!")


Found 94 missing translations.
✅ Missing translations fixed and saved!


In [ ]:
import pandas as pd

# Load the fixed lexicon
df_fixed = pd.read_excel("expanded_lexicon_fixed_v2.xlsx")

# Check for missing English translations
missing_after = df_fixed[df_fixed["English"].isna() | (df_fixed["English"] == "")]
print(f"Still missing after fixing: {len(missing_after)}")

# Optional: see the words that are still missing
if len(missing_after) > 0:
    print(missing_after[["French", "English"]])

Still missing after fixing: 0


In [ ]:
import pandas as pd

# Load the fixed lexicon
df_fixed = pd.read_excel("expanded_lexicon_fixed_v2.xlsx")

# List of columns to check
langs = ["Zulu", "Xhosa", "Sepedi", "Shona", "Afrikaans"]

# Check for missing translations in each language
for lang in langs:
    missing = df_fixed[df_fixed[lang].isna() | (df_fixed[lang] == "")]
    print(f"{lang}: {len(missing)} missing translations")
    if len(missing) > 0:
        print(missing[["French", "English", lang]].head(10))  # show first 10 missing


Zulu: 10 missing translations
          French     English Zulu
447        route        road  NaN
748     pantalon       pants  NaN
765       doigts     fingers  NaN
832      chanter        sing  NaN
1561      Demain    Tomorrow  NaN
4955  chimérique  chimerical  NaN
4959      chiper       steal  NaN
4962      chique      classy  NaN
4963       jabot        crop  NaN
4966    jacasser     chatter  NaN
Xhosa: 13 missing translations
          French     English Xhosa
447        route        road   NaN
748     pantalon       pants   NaN
765       doigts     fingers   NaN
832      chanter        sing   NaN
1561      Demain    Tomorrow   NaN
1570     Abeille         Bee   NaN
4955  chimérique  chimerical   NaN
4957   chimpanzé  chimpanzee   NaN
4959      chiper       steal   NaN
4962      chique      classy   NaN
Sepedi: 112 missing translations
            French      English Sepedi
447          route         road    NaN
520        trouver         find    NaN
748       pantalon        pant

In [ ]:
import pandas as pd
import time
from googletrans import Translator

# Load the fixed lexicon
df = pd.read_excel("expanded_lexicon_fixed_v2.xlsx")

translator = Translator()

# Supported language codes
langs = {
    "Zulu": "zu",
    "Xhosa": "xh",
    "Sepedi": "nso",   # Note: Sepedi sometimes not supported well
    "Shona": "sn",
    "Afrikaans": "af"
}

def safe_translate(text, src="en", dest="zu", retries=3, delay=2):
    """Translate with retries to handle errors/timeouts."""
    if pd.isna(text) or text.strip() == "":
        return None
    for _ in range(retries):
        try:
            result = translator.translate(text, src=src, dest=dest)
            if result and result.text:
                return result.text
        except Exception:
            time.sleep(delay)
    return None

# Translate only missing entries for each language
for lang, code in langs.items():
    missing = df[df[lang].isna() | (df[lang] == "")]
    print(f"\n🔎 Translating {len(missing)} missing entries for {lang}...")
    
    for i, row in missing.iterrows():
        english_word = str(row["English"]).strip()
        if not english_word:
            continue
        translated = safe_translate(english_word, src="en", dest=code)
        df.at[i, lang] = translated if translated else ""

# Save updated file
df.to_excel("expanded_lexicon_v3.xlsx", index=False)
print("\n✅ Finished! Missing translations have been filled.")



🔎 Translating 10 missing entries for Zulu...

🔎 Translating 13 missing entries for Xhosa...

🔎 Translating 112 missing entries for Sepedi...

🔎 Translating 25 missing entries for Shona...

🔎 Translating 27 missing entries for Afrikaans...

✅ Finished! Missing translations have been filled.


In [ ]:
import pandas as pd

# Load the new file
df = pd.read_excel("expanded_lexicon_v3.xlsx")

langs = ["Zulu", "Xhosa", "Sepedi", "Shona", "Afrikaans"]

# Check if any missing values remain
for lang in langs:
    missing = df[df[lang].isna() | (df[lang] == "")]
    print(f"{lang}: {len(missing)} still missing")
    if len(missing) > 0:
        print(missing[["French", "English", lang]].head(10))


Zulu: 0 still missing
Xhosa: 0 still missing
Sepedi: 112 still missing
            French      English Sepedi
447          route         road    NaN
520        trouver         find    NaN
748       pantalon        pants    NaN
765         doigts      fingers    NaN
832        chanter         sing    NaN
887       partager        share    NaN
1561        Demain     Tomorrow    NaN
1570       Abeille          Bee    NaN
1597   Information  Information    NaN
1602          Père       Father    NaN
Shona: 0 still missing
Afrikaans: 0 still missing


In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

# Load your lexicon
df = pd.read_excel("expanded_lexicon_fixed_v2.xlsx")
missing_sepedi = df[df["Sepedi"].isna() | (df["Sepedi"] == "")]
print(f"Found {len(missing_sepedi)} missing Sepedi entries.")

OPENAI_API_KEY = "YOUR_API_KEY_HERE"

def translate_to_sepedi(text):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "gpt-4",
        "messages": [{"role": "user", "content": f"Translate the English word '{text}' to Sepedi (Northern Sotho)."}],
        "temperature": 0
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content'].strip()
    else:
        print(f"⚠️ API error for '{text}': {response.text}")
        return None

# Translate missing entries
for idx, row in tqdm(missing_sepedi.iterrows(), total=len(missing_sepedi)):
    english_word = row["English"]
    df.at[idx, "Sepedi"] = translate_to_sepedi(english_word)

df.to_excel("expanded_lexicon_v4.xlsx", index=False)
print("✅ Sepedi translations completed!")


Found 112 missing Sepedi entries.


  1%|          | 1/112 [00:00<01:34,  1.17it/s]

⚠️ API error for 'road': {
    "error": {
        "message": "Incorrect API key provided: YOUR_API*****HERE. You can find your API key at https://platform.openai.com/account/api-keys.",
        "type": "invalid_request_error",
        "param": null,
        "code": "invalid_api_key"
    }
}



  2%|▏         | 2/112 [00:01<01:28,  1.24it/s]

⚠️ API error for 'find': {
    "error": {
        "message": "Incorrect API key provided: YOUR_API*****HERE. You can find your API key at https://platform.openai.com/account/api-keys.",
        "type": "invalid_request_error",
        "param": null,
        "code": "invalid_api_key"
    }
}



...

 99%|█████████▉| 111/112 [01:24<00:00,  1.32it/s]

⚠️ API error for 'Breathing': {
    "error": {
        "message": "Incorrect API key provided: YOUR_API*****HERE. You can find your API key at https://platform.openai.com/account/api-keys.",
        "type": "invalid_request_error",
        "param": null,
        "code": "invalid_api_key"
    }
}



100%|██████████| 112/112 [01:25<00:00,  1.31it/s]

⚠️ API error for 'Firmly': {
    "error": {
        "message": "Incorrect API key provided: YOUR_API*****HERE. You can find your API key at https://platform.openai.com/account/api-keys.",
        "type": "invalid_request_error",
        "param": null,
        "code": "invalid_api_key"
    }
}



✅ Sepedi translations completed!


In [2]:
pip install deep-translator pandas openpyxl


  Using cached deep_translator-1.11.4-py3-none-any.whl.metadata (30 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached beautifulsoup4-4.14.2-py3-none-any.whl.metadata (3.8 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached soupsieve-2.8-py3-none-any.whl.metadata (4.6 kB)
Using cached deep_translator-1.11.4-py3-none-any.whl (42 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.0 MB 8.3 MB/s eta 0:00:02
   -------- ------------------------------- 2.4/11.0 MB 7.1 MB/s eta 0:00:02
   ------------- -------------------------- 3.7/11.0 MB 6.8 MB/s eta 0:00:02
   ------------------ --------------------- 5.0/11.0 MB 6.9 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/11.0 MB 6.8 MB/s eta 0:00:01
   ----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
from deep_translator import GoogleTranslator




In [10]:
import pandas as pd

input_file = r"C:\Users\visha\OneDrive - University of Pretoria\Desktop\Honours\INF 791\Assignment 3\expanded_lexicon_v4.xlsx"

# Try loading
df = pd.read_excel(input_file)
df.head()


,CILUBA,French,Score,Sentiment,Nature,English,Zulu,Xhosa,Sepedi,Shona,Afrikaans
0,Akaja,Arrange,1.0,Positif,Verbe,Arrange,Hlela,Cwangcisa,Hlophisa,Ronga,Reël
1,Akajilula,Rearrange,1.0,Positif,Verbe,Rear range,Ibanga langemuva,Uluhlu lwangasemva,Boraro ba morao,Kumashure,Grootafstand
2,Akula,Parle,2.0,Positif,Verbe,Speak,Khuluma,Thetha,Bua,Taura,Praat
3,Akulula,Reparle,2.0,Positif,Verbe,Speak again,Khuluma futhi,Thetha kwakhona,Bua hape,Taura zvakare,Praat weer
4,Aluja,Remet,3.0,Positif,Verbe,Hands over,Izandla ngaphezulu,Izandla ngaphezulu,Matsoho a fetang,Maoko pamusoro,Hande om


In [15]:
import pandas as pd

input_file = r"C:\Users\visha\OneDrive - University of Pretoria\Desktop\Honours\INF 791\Assignment 3\expanded_lexicon_v3.xlsx"

# Try loading
df = pd.read_excel(input_file)
df.head()


,CILUBA,French,Score,Sentiment,Nature,English,Zulu,Xhosa,Sepedi,Shona,Afrikaans
0,Akaja,Arrange,1.0,Positif,Verbe,Arrange,Hlela,Cwangcisa,Hlophisa,Ronga,Reël
1,Akajilula,Rearrange,1.0,Positif,Verbe,Rear range,Ibanga langemuva,Uluhlu lwangasemva,Boraro ba morao,Kumashure,Grootafstand
2,Akula,Parle,2.0,Positif,Verbe,Speak,Khuluma,Thetha,Bua,Taura,Praat
3,Akulula,Reparle,2.0,Positif,Verbe,Speak again,Khuluma futhi,Thetha kwakhona,Bua hape,Taura zvakare,Praat weer
4,Aluja,Remet,3.0,Positif,Verbe,Hands over,Izandla ngaphezulu,Izandla ngaphezulu,Matsoho a fetang,Maoko pamusoro,Hande om


In [ ]:
import pandas as pd

# Load the new file
df = pd.read_excel("expanded_lexicon_v3.xlsx")

langs = ["Zulu", "Xhosa", "Sepedi", "Shona", "Afrikaans"]

# Check if any missing values remain
for lang in langs:
    missing = df[df[lang].isna() | (df[lang] == "")]
    print(f"{lang}: {len(missing)} still missing")
    if len(missing) > 0:
        print(missing[["French", "English", lang]].head(10))


Zulu: 0 still missing
Xhosa: 0 still missing
Sepedi: 112 still missing
            French      English Sepedi
447          route         road    NaN
520        trouver         find    NaN
748       pantalon        pants    NaN
765         doigts      fingers    NaN
832        chanter         sing    NaN
887       partager        share    NaN
1561        Demain     Tomorrow    NaN
1570       Abeille          Bee    NaN
1597   Information  Information    NaN
1602          Père       Father    NaN
Shona: 0 still missing
Afrikaans: 0 still missing


In [17]:
from deep_translator import GoogleTranslator

# Function to translate English -> Sepedi
def translate_to_sepedi(text):
    try:
        return GoogleTranslator(source='en', target='nso').translate(text)
    except Exception as e:
        print("Translation error:", e)
        return None

# Fill missing Sepedi words
for idx, row in df[df["Sepedi"].isna() | (df["Sepedi"] == "")].iterrows():
    english_word = row["English"]
    sepedi_translation = translate_to_sepedi(english_word)
    df.at[idx, "Sepedi"] = sepedi_translation
    print(f"{english_word} -> {sepedi_translation}")

# Save updated Excel
df.to_excel("expanded_lexicon_v4.xlsx", index=False)
print("✅ Missing Sepedi words filled and saved.")


road -> tsela
find -> hwetša
pants -> marokgwana
fingers -> Menwana .
sing -> opela
share -> abelana
Tomorrow -> Gosasa
Bee -> Nosi
Information -> Tshedimošo
Father -> Papa
Education -> Thuto
teacher -> morutiši
Sadness -> manyami a go nyama .
Pillow -> Mosamelo
Whirlwind -> Ledimo la ledimo .
Rain -> Pula
thumbtack -> Thumbtack .
Young Woman -> Mosadi yo mofsa .
Youngman -> Youngman .
Uncle -> Malome
Aunt -> Rakgadi
Do -> Dira
Urine -> Moroto .
THANKS -> KE A LEBOGA
Sleep -> Robala
Struck -> Bethilwe
Turn -> Fetoga
Read -> Bala
Eat -> Ja
Disease -> Bolwetši
Dead -> Hlokofetše
Deceased -> Mohu
Pity -> Kwelobohloko
Ants -> Ditšhošwane .
Bird -> Nonyana
Mad -> Gafa
Rich -> Humile
Marry -> Nyala .
Rabbit -> Mmutla
Bike -> Paesekela
Fear -> Tšhoga
Mushrooms -> Di-mushroom .
Hour -> Iri
Sabotaged -> Sabotaged .
Mouth -> Molomo
Mango -> Mango .
lion -> tau
Monkey -> Kgabo
Elephant -> Tlou
bottle -> lebotlelo
Water -> Meetse
Machete -> Machete .
Ax -> Selepe .
Spade -> Spade .
Tarpaulin -> Ta

In [18]:
from deep_translator import GoogleTranslator

# Function to translate English -> Sepedi
def translate_to_sepedi(text):
    try:
        return GoogleTranslator(source='en', target='nso').translate(text)
    except Exception as e:
        print("Translation error:", e)
        return None

# Fill missing Sepedi words
for idx, row in df[df["Sepedi"].isna() | (df["Sepedi"] == "")].iterrows():
    english_word = row["English"]
    sepedi_translation = translate_to_sepedi(english_word)
    df.at[idx, "Sepedi"] = sepedi_translation
    print(f"{english_word} -> {sepedi_translation}")

# Save updated Excel
df.to_excel("expanded_lexicon_v4.xlsx", index=False)
print("✅ Missing Sepedi words filled and saved.")


✅ Missing Sepedi words filled and saved.


In [19]:
langs = ["Zulu", "Xhosa", "Sepedi", "Shona", "Afrikaans"]

# Check for missing values
for lang in langs:
    missing_count = df[lang].isna().sum() + (df[lang] == "").sum()
    if missing_count == 0:
        print(f"✅ {lang} has no missing words")
    else:
        print(f"⚠️ {lang} still has {missing_count} missing words")


✅ Zulu has no missing words
✅ Xhosa has no missing words
✅ Sepedi has no missing words
✅ Shona has no missing words
✅ Afrikaans has no missing words
